In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
import matplotlib.ticker as ticker
import matplotlib.cm as cm
import numpy as np
import csv
from tabulate import tabulate

In [7]:
# Load the dataset

files = ['industrygroup19.csv', 'industrygroup20.csv', 'industrygroup21.csv',
         'industrygroup22.csv']


years = ['2019', '2020', '2021', '2022']


df_list = []


for file, year in zip(files, years):
    df = pd.read_csv(file)
    df.insert(0, 'Year', year)
    df_list.append(df)


combined_df = pd.concat(df_list, ignore_index=True)


combined_df.to_csv('combined_in.csv', index=False)

In [8]:
df = pd.read_csv('combined_in.csv')

In [9]:
df

,Year,Unnamed: 0,Unnamed: 1,UK SIC 2007,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,...,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37
0,2019,Code,Area,"SIC07: 01-03 : Agriculture, forestry & fishing",SIC07: 05-39 : Production,SIC07: 41-43 : Construction,SIC07: 45 : Motor trades,SIC07: 46 : Wholesale,SIC07: 47 : Retail,SIC07: 49-53 : Transport & Storage (inc. postal),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,E09000001,City of London,25,835,785,30,640,"1,425",290,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019,E09000002,Barking and Dagenham,10,335,"1,645",225,405,710,620,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019,E09000003,Barnet,35,610,"3,385",360,"1,205","2,400",635,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022,K03000001,Great Britain,"128,035","165,180","374,415","88,815","123,960","308,130","150,940",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,2022,K02000001,United Kingdom,"146,420","171,205","385,755","91,990","127,650","317,005","154,665",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Load the dataset, skipping the first row
df = pd.read_csv('combined_in.csv', skiprows=1)

columns_to_drop = ['Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 
                   'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 
                   'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38']

df = df.drop(columns=columns_to_drop)

# Save the DataFrame to a new CSV file
df.to_csv('modified.csv', index=False)

In [12]:
filename = "modified.csv"
# Read the CSV file
with open(filename, 'r') as file:
    csv_reader = csv.reader(file)
    rows = list(csv_reader)

# Modify the first line
rows[0][0] = "Year"

# Write the modified content back to the CSV file
with open(filename, 'w', newline='') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerows(rows)

In [13]:
# Read the CSV file into a DataFrame
DF = pd.read_csv('modified.csv')

# Drop rows based on a condition
condition = DF['Area'].isin(['Inner London', 'Outer London', 'North East','North West',
                            'Yorkshire and The Humber','East Midlands','West Midlands','East','London',
                            'South East','South West','England','Wales','Scotland',
                            'Northern Ireland','England and Wales','Great Britain','United Kingdom'])

DF = DF[~condition]
# Save the modified DataFrame back to a CSV file
DF.to_csv('modified_update.csv', index=False)

In [14]:

df1 = pd.read_csv('modified_update.csv')

for col in df1.columns[3:]:
    df1[col][1:] = pd.to_numeric(df1[col][1:].str.replace(',', ''), errors='coerce')

# Drop rows with missing data
df1 = df1.dropna()


/var/folders/3_/wmg86y9x5nd633flp8f52_nh0000gn/T/ipykernel_87811/2666377254.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[col][1:] = pd.to_numeric(df1[col][1:].str.replace(',', ''), errors='coerce')


In [16]:

df_2019 = df1[df1['Year'] == 2019]
sum_2019 = df_2019.drop(['Year', 'Code', 'Area'], axis=1).sum()
df_sum19 = pd.DataFrame(sum_2019).T
df_sum19['Year'] = '2019'
df_sum19['Code'] = 'Total'
df_sum19['Area'] = 'Total'
df_2019 = df_2019.append(df_sum19, ignore_index=True)


df_2020 = df1[df1['Year'] == 2020]
sum_2020 = df_2020.drop(['Year', 'Code', 'Area'], axis=1).sum()
df_sum20 = pd.DataFrame(sum_2020).T
df_sum20['Year'] = '2020'
df_sum20['Code'] = 'Total'
df_sum20['Area'] = 'Total'
df_2020 = df_2020.append(df_sum20, ignore_index=True)


df_2021 = df1[df1['Year'] == 2021]
sum_2021 = df_2021.drop(['Year', 'Code', 'Area'], axis=1).sum()
df_sum21 = pd.DataFrame(sum_2021).T
df_sum21['Year'] = '2021'
df_sum21['Code'] = 'Total'
df_sum21['Area'] = 'Total'
df_2021 = df_2021.append(df_sum21, ignore_index=True)

df_2022 = df1[df1['Year'] == 2022]
sum_2022 = df_2022.drop(['Year', 'Code', 'Area'], axis=1).sum()
df_sum22 = pd.DataFrame(sum_2022).T
df_sum22['Year'] = '2022'
df_sum22['Code'] = 'Total'
df_sum22['Area'] = 'Total'
df_2022 = df_2022.append(df_sum22, ignore_index=True)


/var/folders/3_/wmg86y9x5nd633flp8f52_nh0000gn/T/ipykernel_87811/3555215472.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2019 = df_2019.append(df_sum19, ignore_index=True)
/var/folders/3_/wmg86y9x5nd633flp8f52_nh0000gn/T/ipykernel_87811/3555215472.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2020 = df_2020.append(df_sum20, ignore_index=True)
/var/folders/3_/wmg86y9x5nd633flp8f52_nh0000gn/T/ipykernel_87811/3555215472.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2021 = df_2021.append(df_sum21, ignore_index=True)
/var/folders/3_/wmg86y9x5nd633flp8f52_nh0000gn/T/ipykernel_87811/3555215472.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

In [17]:
df_2019.to_csv('total2019.csv', index=False)
df_2020.to_csv('total2020.csv', index=False)
df_2021.to_csv('total2021.csv', index=False)
df_2022.to_csv('total2022.csv', index=False)

In [20]:

files1 = [ 'total2019.csv', 'total2020.csv', 'total2021.csv', 'total2022.csv']

# Create an empty DataFrame to store the combined data
combined_d = pd.DataFrame()

# Iterate over the files
for file in files1:
    # Read each file into a temporary DataFrame
    temp_df = pd.read_csv(file)
    
    # Append the temporary DataFrame to the combined DataFrame
    combined_d = combined_d.append(temp_df, ignore_index=True)

combined_d.rename(columns=lambda x: x.lower(), inplace=True)
# Save the combined DataFrame to a new CSV file
combined_d.to_csv('modicombine.csv', index=False)


/var/folders/3_/wmg86y9x5nd633flp8f52_nh0000gn/T/ipykernel_87811/1200015663.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_d = combined_d.append(temp_df, ignore_index=True)
/var/folders/3_/wmg86y9x5nd633flp8f52_nh0000gn/T/ipykernel_87811/1200015663.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_d = combined_d.append(temp_df, ignore_index=True)
/var/folders/3_/wmg86y9x5nd633flp8f52_nh0000gn/T/ipykernel_87811/1200015663.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_d = combined_d.append(temp_df, ignore_index=True)
/var/folders/3_/wmg86y9x5nd633flp8f52_nh0000gn/T/ipykernel_87811/1200015663.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas i

In [21]:
DF = pd.read_csv('modicombine.csv')
DF

,year,code,area,"sic07: 01-03 : agriculture, forestry & fishing",sic07: 05-39 : production,sic07: 41-43 : construction,sic07: 45 : motor trades,sic07: 46 : wholesale,sic07: 47 : retail,sic07: 49-53 : transport & storage (inc. postal),...,sic07: 58-63 : information & communication,sic07: 64-66 : finance & insurance,sic07: 68 : property,"sic07: 69-75 : professional, scientific & technical",sic07: 77-82 : business administration & support services,sic07: 84 : public administration & defence,sic07: 85 : education,sic07: 86-88 : health,"sic07: 90-99 : arts, entertainment, recreation & other services",sic07: total
0,2019,E09000001,City of London,25.0,835.0,785.0,30.0,640.0,1425.0,290.0,...,2415.0,3700.0,1095.0,8040.0,5225.0,55.0,215.0,335.0,1070.0,27460.0
1,2019,E09000002,Barking and Dagenham,10.0,335.0,1645.0,225.0,405.0,710.0,620.0,...,665.0,90.0,140.0,790.0,790.0,170.0,195.0,495.0,380.0,8095.0
2,2019,E09000003,Barnet,35.0,610.0,3385.0,360.0,1205.0,2400.0,635.0,...,2690.0,535.0,2145.0,4955.0,2270.0,55.0,540.0,1230.0,1610.0,25660.0
3,2019,E09000004,Bexley,10.0,450.0,1980.0,270.0,330.0,815.0,405.0,...,1090.0,185.0,225.0,1490.0,855.0,25.0,230.0,490.0,555.0,9985.0
4,2019,E09000005,Brent,5.0,575.0,2400.0,405.0,955.0,1895.0,690.0,...,1900.0,270.0,775.0,2650.0,1375.0,65.0,315.0,745.0,1035.0,16990.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,2022,E09000030,Tower Hamlets,15.0,620.0,950.0,175.0,865.0,1740.0,595.0,...,2765.0,550.0,995.0,3605.0,2000.0,65.0,560.0,730.0,1305.0,19065.0
132,2022,E09000031,Waltham Forest,25.0,505.0,2380.0,235.0,575.0,1475.0,680.0,...,1170.0,120.0,395.0,1940.0,1175.0,20.0,255.0,560.0,965.0,13275.0
133,2022,E09000032,Wandsworth,30.0,515.0,1195.0,150.0,720.0,1555.0,390.0,...,1930.0,300.0,840.0,4270.0,1935.0,40.0,420.0,860.0,1505.0,17845.0
134,2022,E09000033,Westminster,125.0,1585.0,3375.0,180.0,1970.0,5605.0,720.0,...,5760.0,4480.0,5475.0,12725.0,6645.0,280.0,785.0,1870.0,4930.0,60510.0


In [25]:
dff = pd.read_csv('modicombine.csv')

dff.drop('sic07: total', axis=1, inplace=True)

# Optionally, you can write the modified DataFrame back to a CSV file
dff.to_csv('bizindustry_cleaned.csv', index=False)

In [26]:
# Filter rows with Code == Total
Total_DF = dff[dff['code'] == 'Total']
Total_DF

,year,code,area,"sic07: 01-03 : agriculture, forestry & fishing",sic07: 05-39 : production,sic07: 41-43 : construction,sic07: 45 : motor trades,sic07: 46 : wholesale,sic07: 47 : retail,sic07: 49-53 : transport & storage (inc. postal),sic07: 55-56 : accommodation & food services,sic07: 58-63 : information & communication,sic07: 64-66 : finance & insurance,sic07: 68 : property,"sic07: 69-75 : professional, scientific & technical",sic07: 77-82 : business administration & support services,sic07: 84 : public administration & defence,sic07: 85 : education,sic07: 86-88 : health,"sic07: 90-99 : arts, entertainment, recreation & other services"
33,2019,Total,Total,780.0,17695.0,57295.0,7395.0,21735.0,50845.0,15595.0,33070.0,73850.0,17965.0,27585.0,124145.0,58525.0,1925.0,11275.0,24130.0,40375.0
67,2020,Total,Total,785.0,16845.0,57450.0,6725.0,19300.0,40125.0,15585.0,24350.0,72510.0,15960.0,25440.0,120460.0,51980.0,100.0,8910.0,18530.0,36245.0
101,2021,Total,Total,835.0,18485.0,60780.0,7795.0,22755.0,56455.0,20225.0,34840.0,69870.0,17755.0,28935.0,119340.0,57580.0,1820.0,12095.0,24870.0,40790.0
135,2022,Total,Total,910.0,18805.0,63210.0,8075.0,23715.0,58020.0,20590.0,36035.0,65175.0,17235.0,30280.0,113830.0,58205.0,1810.0,12650.0,25365.0,41700.0


In [27]:
Total_DF.to_csv('Total_by_each_Industry.csv', index=False)